In [42]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import re
from tqdm import tqdm
import random
import os

# 열의 최대 너비 설정
pd.set_option('display.max_colwidth', None)

# 최대 표시 행 수 설정
pd.set_option('display.max_rows', None)

api_key = "64a1bdc693984439b9e3a23e58ea5162"

In [43]:
def get_sources(api_key, language="en"):
    base_url = "https://newsapi.org/v2/sources"
    parameters = {
        "language": language,
        "apiKey": api_key
    }
    response = requests.get(base_url, params=parameters)
    if response.status_code == 200:
        return [source['id'] for source in response.json()['sources']]
    else:
        return []

In [38]:
import time

def get_news(api_key, start_date, end_date, query="bitcoin", language="en", page_size=100, max_pages=5, delay=2):
    base_url = "https://newsapi.org/v2/everything"
    all_articles = []
    
    for page in range(1, max_pages + 1):
        parameters = {
            "q": query,
            "language": language,
            "pageSize": page_size,
            "page": page,
            "apiKey": api_key,
            "from": start_date,
            "to": end_date,
            "sortBy": "publishedAt",
            "sources" : 'cnn'
        }

        response = requests.get(base_url, params=parameters)
        
        if response.status_code == 200:
            articles = response.json().get('articles', [])
            all_articles.extend(articles)
        else:
            print(f"Error on page {page}: {response.status_code}")
        
        if page != max_pages:  # 마지막 페이지를 제외하고 대기합니다.
            time.sleep(delay)
    
    return {"articles": all_articles}


In [37]:
def get_news(api_key, start_date, end_date, query="bitcoin", language="en", page_size=100, max_pages=5):
    base_url = "https://newsapi.org/v2/everything"
    all_articles = []
    
    for page in range(1, max_pages + 1):
        parameters = {
            "q": query,
            "language": language,
            "pageSize": page_size,
            "page": page,
            "apiKey": api_key,
            "from": start_date,
            "to": end_date,
            "sortBy": "publishedAt",
            "sources" : 'cnn'
            
        }

        response = requests.get(base_url, params=parameters)
        
        if response.status_code == 200:
            articles = response.json().get('articles', [])
            all_articles.extend(articles)
        else:
            print(f"Error on page {page}: {response.status_code}")
    
    return {"articles": all_articles}


In [14]:
def convert_to_dataframe(news_data):
    if news_data and "articles" in news_data:
        df = pd.DataFrame(news_data['articles'])
        return df
    else:
        return pd.DataFrame()

In [25]:
# from newspaper import Article

# def get_full_content(url):
#     try:
#         article = Article(url)
#         article.download()
#         article.parse()
#         return article.text
#     except Exception as e:
#         print(f"Error with URL {url}: {e}")
#         return None

# # df['url']를 사용하여 각 URL에 대한 전체 본문 내용을 가져옵니다.
# df['full_content'] = df['url'].apply(get_full_content)


Error with URL https://hivemind.vc/nostr/: Article `download()` failed with 429 Client Error: Too Many Requests for url: https://hivemind.vc/nostr/ on URL https://hivemind.vc/nostr/
Error with URL https://www.forbes.com/sites/digital-assets/2023/09/01/bitcoin-loses-weekly-gains-after-sec-postpones-spot-etf-ruling/: Article `download()` failed with 403 Client Error: Max restarts limit reached for url: https://www.forbes.com/sites/digital-assets/2023/09/01/bitcoin-loses-weekly-gains-after-sec-postpones-spot-etf-ruling/ on URL https://www.forbes.com/sites/digital-assets/2023/09/01/bitcoin-loses-weekly-gains-after-sec-postpones-spot-etf-ruling/
Error with URL https://www.forbes.com/sites/owentedford/2023/09/01/gary-genslers-crypto-approach-likely-wont-be-slowed-by-the-courts/: Article `download()` failed with 403 Client Error: Max restarts limit reached for url: https://www.forbes.com/sites/owentedford/2023/09/01/gary-genslers-crypto-approach-likely-wont-be-slowed-by-the-courts/ on URL htt

In [20]:
sources = get_sources(api_key)

In [41]:
len(sources)

81

In [39]:
start_date = '2023-09-01'
end_date = '2023-09-26'

news_data = get_news(api_key, start_date, end_date)
df = convert_to_dataframe(news_data)
print(df)

Error on page 2: 426
Error on page 3: 426
Error on page 4: 426
Error on page 5: 426
                         source          author  \
0  {'id': 'cnn', 'name': 'CNN'}  Allison Morrow   

                                                     title  \
0  Bankrupt FTX gets permission to liquidate crypto assets   

                                                                                                                                                  description  \
0  A bankruptcy judge granted FTX, the collapsed crypto exchange founded by Sam Bankman-Fried, permission to liquidate its digital assets to repay creditors.   

                                                                              url  \
0  https://www.cnn.com/2023/09/13/business/ftx-liquidate-crypto-assets/index.html   

                                                                                                                          urlToImage  \
0  https://media.cnn.com/api/v1/images/stellar/prod/230913

In [36]:
df.to_csv('crypto_news_api_time.csv', encoding='utf-8-sig')

In [40]:
df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'cnn', 'name': 'CNN'}",Allison Morrow,Bankrupt FTX gets permission to liquidate crypto assets,"A bankruptcy judge granted FTX, the collapsed crypto exchange founded by Sam Bankman-Fried, permission to liquidate its digital assets to repay creditors.",https://www.cnn.com/2023/09/13/business/ftx-liquidate-crypto-assets/index.html,"https://media.cnn.com/api/v1/images/stellar/prod/230913154745-04-sam-bankman-fried-manhattan-court-0726.jpg?c=16x9&q=w_800,c_fill",2023-09-13T20:13:15Z,"A bankruptcy judge granted FTX, the collapsed crypto exchange founded by Sam Bankman-Fried, permission to liquidate its digital assets to repay creditors. \r\nFTX has previously said in bankruptcy cour… [+949 chars]"


In [30]:
# # df는 이전에 생성한 뉴스 데이터 프레임입니다.
# df['content'] = df['url'].apply(extract_article_content)
# print(df[['title', 'content']])

                                               title  \
0  SEC Reportedly Asked Coinbase to Halt All Trad...   
1  Everyone's Favorite Knife-Wielding Robot Retur...   
2  PayPal launches PYUSD stablecoin backed by the...   
3  First Bitcoin ETF Could Be Coming Soon as Cour...   
4  Before suing Coinbase, SEC asked it to trade o...   
5  Binance Australia customers seen selling bitco...   
6  Crypto stocks dip after bitcoin slumps to six-...   
7  Bitcoin plunges more than 10% after Elon Musk'...   
8  Bitcoin could drop to the $20,000 range if a k...   
9  Bitcoin is headed to $180,000 if regulators ap...   

                                             content  
0  Coinbase’s CEO Brian Armstrong reportedly told...  
1  Futurama’s new season continues its examinatio...  
2  PayPal is launching its own stablecoin: PayPal...  
3  The U.S. Court of Appeals for the D.C. Circuit...  
4  A smartphone with displayed Coinbase logo and ...  
5  The Binance logo is seen in this illustration ... 

In [26]:
url = "https://markets.businessinsider.com/news/currencies/bitcoin-price-crypto-token-august-stocks-bonds-equities-markets-investors-2023-9"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

content_lock_content = soup.select_one('.content-lock-content')

if content_lock_content:
    print(content_lock_content.text)
else:
    print("No content found for the given selector.")



Bitcoin finished August about 10% lower, its second consecutive losing month. 
Crypto was caught up in a tough sell-off that hit stocks and bonds throughout the month. 
Bitcoin extended losses into September, down more than 2.5% on Friday to trade at $25,660. 











 
NEW LOOK


                                    Sign up to get the inside scoop on today’s biggest stories in markets, tech, and business — delivered daily. Read preview








 


Loading
Something is loading.













Thanks for signing up!


                              Access your favorite topics in a personalized feed while you're on the go.
                              
                                download the app
                               





Email address





                                      Sign up
                                     



                                  By clicking “Sign Up”, you accept our Terms of Service and Privacy Policy. You can opt-out at any time.
         

In [29]:
content_lock_content.text

'\n\nBitcoin finished August about 10% lower, its second consecutive losing month.\xa0\nCrypto was caught up in a tough sell-off that hit stocks and bonds throughout the month.\xa0\nBitcoin extended losses into September, down more than 2.5% on Friday to trade at $25,660.\xa0\n\n\n\n\n\n\n\n\n\n\n\n \nNEW LOOK\n\n\n                                    Sign up to get the inside scoop on today’s biggest stories in markets, tech, and business — delivered daily. Read preview\n\n\n\n\n\n\n\n\n \n\n\nLoading\nSomething is loading.\n\n\n\n\n\n\n\n\n\n\n\n\n\nThanks for signing up!\n\n\n                              Access your favorite topics in a personalized feed while you\'re on the go.\n                              \n                                download the app\n                               \n\n\n\n\n\nEmail address\n\n\n\n\n\n                                      Sign up\n                                     \n\n\n\n                                  By clicking “Sign Up”, you accep